In [1]:
from pathlib import Path
import gzip
import pickle
import requests
import torch

In [2]:
url = "https://github.com/pytorch/tutorials/raw/master/_static/"
filename = "mnist.pkl.gz"

In [3]:
data_path = Path("data") if Path("data").exists() else Path("../data")
path = data_path / "downloaded"/"vector_mnist"
path.mkdir(parents=True,exist_ok=True)

In [4]:
if not (path/filename).exists():
    content = requests.get(url+filename).content
    (path/filename).open("wb").write(content) 

In [5]:
!cd ../ && tree

.
├── data
│   └── downloaded
│       └── vector_mnist
│           └── mnist.pkl.gz
├── notebooks
│   ├── lab01_pytorch.ipynb
│   └── lab01_pytorch_self.ipynb
└── text_recognizer
    ├── __init__.py
    ├── __pycache__
    │   ├── __init__.cpython-310.pyc
    │   └── __init__.cpython-38.pyc
    ├── data
    │   └── util.py
    ├── metadata
    │   ├── __pycache__
    │   │   ├── mnist.cpython-310.pyc
    │   │   ├── mnist.cpython-38.pyc
    │   │   ├── shared.cpython-310.pyc
    │   │   └── shared.cpython-38.pyc
    │   ├── mnist.py
    │   └── shared.py
    ├── models
    │   ├── __init__.py
    │   └── mlp.py
    └── util.py

10 directories, 16 files


In [6]:
datafile = (path/filename)

In [7]:
with gzip.open(datafile,"rb") as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f,encoding="latin-1")

In [8]:
x_train, y_train, x_valid, y_valid = map(torch.Tensor,
(x_train, y_train,x_valid, y_valid))

In [9]:
x_train[0,::2].shape

torch.Size([392])

In [10]:
x_train.ndim

2

In [11]:
# re-execute this cell for more samples
import sys
sys.path.append("/Users/convergeml/engineering/fsdl-text-recognizer-2022-labs/lab01")

import random
import wandb  # just for some convenience methods that convert tensors to human-friendly datatypes
import text_recognizer.metadata.mnist as metadata # metadata module holds metadata separate from data

In [12]:
idx = random.randint(0, len(x_train))
example = x_train[idx]

print(y_train[idx])  # the label of the image
wandb.Image(example.reshape(1,28,28)).image  # the image itself

tensor(1.)


In [13]:
import math

In [16]:
weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10,requires_grad=True)

In [17]:
def linear(x: torch.Tensor) -> torch.Tensor:
    return x @ weights + bias

In [22]:
def logsoftmax(x: torch.Tensor):
    return x - torch.log(torch.sum(torch.exp(x),axis=1))[:,None]

In [23]:
def model(x: torch.Tensor):
    return logsoftmax(linear(x))

In [26]:
#64*784 , 784 * 10
#64 * 10, 10
#64 * 10

torch.Size([64, 784])

In [24]:
bs=64
xb = x_train[0:bs]
outs = model(xb)

print(outs[0],outs.shape)

tensor([-2.7050, -2.3431, -2.2415, -2.3054, -2.4837, -2.8456, -2.2032, -1.7082,
        -2.0582, -2.6560], grad_fn=<SelectBackward0>) torch.Size([64, 10])
